# Optimizing the number of staff in a chain of stores

[![Advertising_campaign_colab.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/colab.ampl.com/blob/master/authors/mikhail/Advertisement/Advertising_campaign_colab.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/colab.ampl.com/blob/master/authors/mikhail/Advertisement/Advertising_campaign_colab.ipynb) [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/colab.ampl.com/blob/master/authors/mikhail/Advertisement/Advertising_campaign_colab.ipynb) [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/colab.ampl.com/blob/master/authors/mikhail/Advertisement/Advertising_campaign_colab.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/colab.ampl.com/blob/master/authors/mikhail/Advertisement/Advertising_campaign_colab.ipynb) [![Hits](https://h.ampl.com/https://github.com/ampl/colab.ampl.com/blob/master/authors/mikhail/Advertisement/Advertising_campaign_colab.ipynb)](https://colab.ampl.com)


[*Partner with the AMPL team to transform complex problems into optimized solutions. AMPL consulting services combine deep technical knowledge with industry-leading insights, helping you unlock the full potential of optimization within your organization.*](https://ampl.com/services/)

Tags: Oil production, Production optimization, Profitability, Refinery, Deterministic model, Piecewise-linear, mip, ampl-only, cbc

Notebook author: Mikhail Riabtsev <<mail@solverytic.com>>
***

## 2. Problem Description

> ### Stage 1: Distillation

An 

> ### Objective

### [How should the operations of the refinery be planned in order to maximize total profit?]()
***

Let's start by downloading the necessary extensions and libraries

In [1]:
# Install dependencies
%pip install -q amplpy pandas 
import pandas as pd                 # Loading panda to work with pandas.DataFrame objects (https://pandas.pydata.org/)
import numpy as np                  # Loading numpy to perform multidimensional calculations numpy.matrix (https://numpy.org/)
import matplotlib.pyplot as plt     # Loading library for creating static, animated, and interactive visualizations (https://matplotlib.org/)


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Google Colab & Kaggle integration # The only 3 lines you need to install and use AMPL with any solver on Colab
'''
from amplpy import AMPL, tools      # Load the library/module for working with AMPL and connect additional modules
ampl = tools.ampl_notebook(
    modules=["cbc", "highs"],       # Pick from over 20 modules including most commercial and open-source solvers ["cbc", "coin", "highs"...]
    license_uuid="default",         # Use your AMPL Community Edition License UUID to gain access to commercial solver trials
    g=globals())                    # Instantiate AMPL object and register magics
    '''

'\nfrom amplpy import AMPL, tools      # Load the library/module for working with AMPL and connect additional modules\nampl = tools.ampl_notebook(\n    modules=["cbc", "highs"],       # Pick from over 20 modules including most commercial and open-source solvers ["cbc", "coin", "highs"...]\n    license_uuid="default",         # Use your AMPL Community Edition License UUID to gain access to commercial solver trials\n    g=globals())                    # Instantiate AMPL object and register magics\n    '

In [3]:
from amplpy import AMPL
from amplpy import add_to_path                      # Загружаем библиотеку/модуль работы с AMPL и задаем путь папки установленного на компьютере AMPL 
add_to_path(r"D:\AMPL\amplide.mswin64\ampl.mswin64")# (r"full path to the AMPL installation directory")"""
ampl = AMPL()                                       # create a new AMPL object with all default settings

## 3. AMPL formulation of the model

>Use %%ampl_eval to evaluate AMPL commands and declarations

In [5]:
%%ampl_eval
reset ;
### SETS
  set PHARMACIES;       # Set of all pharmacies
  set EMPLOYEES;        # Set of all employees
  set MODES;            # Set of staff work modes
  set YEAR_WEEKS;       # Set of weeks in a year
  set DAYS;             # Set of days per week
  set HOURS;            # Set of hours in a day
  set WORKPLACE within {EMPLOYEES, PHARMACIES}; # Base place of work of the staff
  set LINKS within {PHARMACIES, YEAR_WEEKS, DAYS, HOURS}; 
  set LINKS_Y = setof{(p,yw,d,h) in LINKS}(p,yw,d);

### PARAMETERS
 ## General
  # Pharmacies
  param pharm_ItemsSold{LINKS};                 # Number of items sold at each pharmacy at each hour on each day
  param pharm_StartWorkingTime{PHARMACIES};     # Start time of work for each pharmacy
  param pharm_EndWorkingTime{PHARMACIES};       # End time of work for each pharmacy
  param pharm_Distance{PHARMACIES, PHARMACIES}; # Distance between pharmacies
  param reserveFactor = 0.25;                   # Reserve personnel factor for months 4 to 6

  # Employees
  param empl_WorkPlace{WORKPLACE};              # Base place of work of the staff
  param empl_MaxReplaceDistance{EMPLOYEES};     # Maximum distance between pharmacies between which staff can be replaced
  param empl_MaxItemsSoldPerHour{EMPLOYEES};    # Maximum number of items an employee can sell per hour based on their qualifications
  param empl_Salary{EMPLOYEES};                 # Established salary for each employee
  param empl_VacationDuration{EMPLOYEES};       # Total vacation duration for each employee
  param empl_VacationStartPeriod{EMPLOYEES, YEAR_WEEKS, DAYS};    # Planned start of vacation periods
  param empl_WeeklyMaxWorkDays = 6;             # Maximum number of working days per week (for Basic, Rotational and Additional modes)
  param empl_VacationPeriod = 24;               # Vacation days per year
 
  ## Basic mode
  param basic_MinDailyWorkHours = 4;            # Minimum duration of the working day
  param basic_DailyWorkHours = 8;               # Duration of the working day
  param basic_MaxWeeklyHours = 40;              # Maximum weekly working hours
  param basic_MaxWeeklyOvertimeHours = 8;       # Maximum permitted additional hours of work per week for an employee
  param basic_MaxYearOvertimeHours = 96;        # Maximum overtime hours for an employee per year

 ## Rotational mode
  param rotat_DailyWorkHours = 14;              # Working hours per day
  param rotat_DaysPerCycle = 12;                # Working days per cycle
  param rotat_Cycle = 14;                       # Days in a rotational cycle
  param rotat_Rest = 14;                        # Rest days after a rotational cycle
  param rotat_MaxWeeklyOvertimeHours = 0;       # Maximum permitted additional hours of work per week for an employee

 ## Additional
  param add_WorkHoursPerDay = 4;                # Duration of the working day
  param add_MaxWeeklyOvertimeHours = 0;         # Maximum permitted additional hours of work per week for an employee

### VARIABLES

var x{PHARMACIES, EMPLOYEES, YEAR_WEEKS, DAYS, HOURS} >= 0; # Basic Mode
var y{PHARMACIES, EMPLOYEES, YEAR_WEEKS, DAYS, HOURS} >= 0; # Shift Mode
var z{PHARMACIES, EMPLOYEES, YEAR_WEEKS, DAYS, HOURS} >= 0; # Additional Mode

  var X{EMPLOYEES, LINKS, MODES} binary;  # 1 if employee e is working at pharmacy p at hour h on day d in year y, 0 otherwise
  var Y{EMPLOYEES, LINKS_Y} binary;       # 1 если работник назначен в день d на работу в аптеку 
  var XX{MPLOYEES, LINKS, MODES} binary   # 1 if employee e is working additional time at pharmacy p at hour h on day d in year y, 0 otherwise
  var Z{EMPLOYEES,YEAR_WEEKS,DAYS} binary; # If employee e is on vacation
  var r{EMPLOYEES, LINKS, MODES} binary;  # If employee e is a reserve personnel in pharmacy p during weeks and days w

### OBJECTIVE FUNCTION
  # Minimize the total cost of personnel while meeting operational requirements
  minimize Total_Cost:
    sum{e in EMPLOYEES, (p,yw,d,h) in LINKS, m in MODES} wage[e] * x[e,p,yw,d,h,m];
        
### CONSTRAINTS
  subject to
 ## Basic Mode Constraints
  # Total weekly working time of each employee in basic mode should not exceed 40 hours
  Basic_Weekly_Hours {e in EMPLOYEES, yw in YEAR_WEEKS}:
    sum{(p,yw,d,h) in LINKS} X[e,p,yw,d,h,'Basic'] <= basic_mode_MaxHours;

  # Each employee in basic mode should have no more than 6 working days per week 
  Basic_Max_Working_Days {e in EMPLOYEES, yw in YEAR_WEEKS}:
    sum{(p,yw,d) in LINKS_Y} Y[e,p,yw,d] <= DaysPerCycle;
    
  # Each employee can work an additional 8 hours per week if necessary
  Basic_Additional_Hours_Weekly {e in EMPLOYEES, yw in YEAR_WEEKS}:
    sum{p in PHARMACIES, h in HOURS, d in DAYS} x[e,p,h,d,w,1..9] * y[e,'Basic'] <= 48;
  
  # Each employee can work no more than 96 additional hours per year
  Basic_Additional_Hours_Yearly {e in EMPLOYEES}:
    sum{p in PHARMACIES, h in HOURS, d in DAYS, w in WEEKS} max(0, x[e,p,h,d,w,1..9] * y[e,'Basic'] - MaxHours) <= MaxOvertimeYear;

  
  # Each employee in basic mode must work at least 4 hours per day
  Basic_Min_Daily_Hours {e in EMPLOYEES, p in PHARMACIES, d in DAYS, w in WEEKS}:
    sum{h in HOURS} x[e,p,h,d,w,1..9] * y[e,'Basic'] >= 4;
    
  # Each employee in basic mode can be assigned to work only once per day
  Basic_Single_Assignment_Per_Day {e in EMPLOYEES, d in DAYS, w in WEEKS}:
    sum{p in PHARMACIES, h in HOURS} x[e,p,h,d,w,1..9] * y[e,'Basic'] <= 1;

  # Ensure 0.5 hours of joint work for case transfer when changing employees  
  Joint_Work_For_Case_Transfer {e1 in EMPLOYEES, e2 in EMPLOYEES, p in PHARMACIES, h in HOURS, d in DAYS, w in WEEKS}:
    x[e1,p,h,d,w,1..9] + x[e2,p,h,d,w,1..9] >= 0.5 * y[e1,'Basic'] * y[e2,'Basic'] * (1 - (e1 = e2));
    
 ## Shift Mode Constraints
  # Each employee in shift mode must work at least 14 hours per day
  Shift_Min_Daily_Hours {e in EMPLOYEES, p in PHARMACIES, d in DAYS, w in WEEKS, m in MONTHS}:
    sum{h in HOURS} x[e,p,h,d,w,m] * y[e,'Shift'] >= ShiftHoursPerDay;
    
  # Each employee in shift mode works 14 hours a day for 14 days
  Shift_Working_Days_And_Rest {e in EMPLOYEES, w in WEEKS, m in MONTHS}:
    sum{p in PHARMACIES, h in HOURS, d in 1..14} x[e,p,h,d,w,m] * y[e,'Shift'] <= 84;

  # Each employee in shift mode must be given a rest for 14 days after working for 14 days
  Shift_Rest_Period {e in EMPLOYEES, w in WEEKS, m in MONTHS}:
    sum{p in PHARMACIES, h in HOURS, d in 15..28} x[e,p,h,d,w,m] * y[e,'Shift'] = 0;
    
  # Each employee must be given a vacation of 24 days
  Shift_Vacation {e in EMPLOYEES}:
    sum{p in PHARMACIES, h in HOURS, d in DAYS, w in WEEKS, m in 1..24} x[e,p,h,d,w,m] * y[e,'Shift'] = 0;

## General Constraints
  # Ensure each employee can process no more than 50 checks per hour
  Employee_Capacity {p in PHARMACIES, h in HOURS, d in DAYS, w in WEEKS, m in MONTHS}:
    sum{e in EMPLOYEES} x[e,p,h,d,w,m] <= max_checks_per_hour / num_checks[p,h,d,w,m];
    
  # Ensure an employee's new place of work is within a radius of no more than 5 km from their main place of work
  Geographical_Constraints {e in EMPLOYEES, p1 in PHARMACIES, p2 in PHARMACIES}:
    distance[p1, p2] <= 5;

  # Provide a 25% reserve of personnel in case of employee illness during the 4th to 6th month
  Personnel_Reserve {e in EMPLOYEES, p in PHARMACIES, h in HOURS, d in DAYS, w in WEEKS, m in 4..6}:
    sum{t in DAYS} x[e,p,h,t,w,m] >= 1.25 * sum{t in DAYS} x[e,p,h,t,w,1..3];
 

 ## Mode Assignment Constraints
  # Ensure each employee is assigned to one operating mode
  Mode_Assignment_Constraints {e in EMPLOYEES}:
    sum{m in MODES} y[e,m] = 1;          

	line 51 offset 2735
	WEEKS is not defined
Error:
	line 51 offset 2741
	syntax error

For support/feedback go to https://discuss.ampl.com or e-mail <support@ampl.com>

	line 56 offset 3006
	WEEKS is not defined
Error:
	line 56 offset 3013
	syntax error

For support/feedback go to https://discuss.ampl.com or e-mail <support@ampl.com>

	line 60 offset 3230
	WEEKS is not defined
Error:
	line 60 offset 3236
	syntax error

For support/feedback go to https://discuss.ampl.com or e-mail <support@ampl.com>

	line 64 offset 3443
	WEEKS is not defined
Error:
	line 64 offset 3449
	syntax error

For support/feedback go to https://discuss.ampl.com or e-mail <support@ampl.com>

	line 68 offset 3718
	WEEKS is not defined
Error:
	line 68 offset 3724
	syntax error

For support/feedback go to https://discuss.ampl.com or e-mail <support@ampl.com>

	line 73 offset 4002
	WEEKS is not defined
Error:
	line 73 offset 4011
	syntax error

For support/feedback go to https://discuss.ampl.com or e-mail <support@amp

AMPLException: line 98 offset 5444
syntax error

## 4. Load data

In [84]:
%%ampl_eval

data;
### SETS
 # Set of crude oil types
  set CRUD = 'Crude1','Crude2' ;  
        
 # Set of distillation products
  set DIST:= 'Light Naphta','Medium Naphta','Heavy Naphta','Light Oil','Heavy Oil','Residuum' ;

 # Set of reforming products
  set REF:= 'Reformed gasoline' ; 

 # Set of cracking products
  set CRACK:= 'Cracked oil','Cracked gasoline' ;

 # Set of Lubricating products
  set LUBR:= 'Lube Oil';

 # Set of final products
  set PROD:= 'Premium Gasoline','Regular Gasoline','Jet Fuel','Fuel Oil','Lube Oil' ;

***

## 5. Solve problem

>Use %%ampl_eval to evaluate AMPL commands and declarations

In [85]:
%%ampl_eval
option solver highs;          # Choosing a solver

# Defining Output Settings 
option show_stats 1;        # (1) Show statistical information about the size of the problem. Default 0 (statistics are not displayed)
option display_1col 0;      # Data Display Settings
option omit_zero_rows 1;    # Hide rows with 0 values. Default (0)

solve;                      # Solve the model


Presolve eliminates 15 constraints.
Substitution eliminates 107 variables.
Adjusted problem:
1259 variables:
	372 binary variables
	96 nonlinear variables
	791 linear variables
1414 constraints; 13987 nonzeros
	99 nonlinear constraints
	1315 linear constraints
	97 equality constraints
	1259 inequality constraints
	58 range constraints
1 nonlinear objective; 899 nonzeros.

HiGHS 1.6.0:HiGHS 1.6.0: optimal solution; objective 685848.772
26660 simplex iterations
19 branching nodes


## 6. Display the solution

>Use %%ampl_eval

In [59]:
%%ampl_eval

display Plant_Working, Crude_Supply, Distill_to_Reforming, Distill_to_Cracking, Distill_to_Lubricating,  Blending, OctaneNumberMin, VaporPressure_Max, Storage_Fraction, Demand, Loan_In, Loan_Out, CashFlow, Waste_Pollutant;

printf {p in PROD, t in 1..nPeriod}: "Octane number for products: %u %s %6.2f\n", t, p, sum{(i,p) in BLENDING} Blending[i,p,t] * Intermed_Octane[i] / sum{(ii,p) in BLENDING}(if Blending[ii,p,t] = 0 then 1 else Blending[ii,p,t]);

printf {p in PROD, t in 1..nPeriod: p='Jet Fue'}: "Vapor pressure limits of product Jet Fuel: %u %s %6.2f\n", t, p, sum{(i,p) in BLENDING} Blending[i,p,t] * Intermed_VaporPressure[i] / (if sum{(ii,p) in BLENDING} Blending[ii,p,t] = 0 then 1 else sum{(ii,p) in BLENDING}Blending[ii,p,t]) ;

printf {t in 1..nPeriod}: "Premium / Regular percentage: %u %6.2f\n", t, 
sum{(i,p) in BLENDING: p ='Premium Gasoline'} Blending[i,p,t] / (if sum{(ii,pp) in BLENDING: pp ='Regular Gasoline'}Blending[ii,pp,t] = 0 then 1 else sum{(ii,pp) in BLENDING: pp ='Regular Gasoline'}Blending[ii,pp,t]) ;

printf {(i,p) in BLENDING, t in 1..nPeriod: p='Fuel Oil'}: "Production Fuel Oil ratio: %u   %s %6.2f\n", t, i, 
    Blending[i,p,t] / (if Blending['Residuum',p,t] = 0 then 1 else Blending['Residuum',p,t]);

Plant_Working [*] :=
 1 1    2 1    4 1    5 1    6 1    7 1    8 1    9 1   10 1   11 1   12 1
;

:                                            Crude_Supply      :=
D1                  Crude1             1     1302.75
D1                  Crude1             2    17845.2
D1                  Crude1             4     4749.83
D1                  Crude1             5     8528.38
D1                  Crude1             6     5394.44
D1                  Crude1             7     3205.38
D1                  Crude1             8     5826.38
D1                  Crude1             9     5272.35
D1                  Crude2             1    25000
D1                  Crude2             2    25000
D1                  Crude2             4    25000
D1                  Crude2             5    25000
D1                  Crude2             6    25000
D1                  Crude2             7    25000
D1                  Crude2             8    25000
D1                  Crude2             9    25000
D1          

## 7. Further steps
Here is a list of steps required to successfully implement the results of mathematical optimization into the refinery production:

- **Verify Solution Accuracy**
  - Check Constraints
  - Validate Assumptions
  - Cross-Check with Historical Data

- **Sensitivity Analysis**
  - Analyze how changes in key parameters (e.g., crude oil prices, demand forecasts) affect the solution

- **Scenario Analysis**
  - Evaluate the solution under different scenarios (e.g., market fluctuations, supply disruptions)

- **Risk Management**
  - Identify Potential Risks: Operational, Market, External

- **Monitoring and Feedback**
  - Implementation Monitoring
  - Continuous Improvement

- **Review and Update**
  - Post-Implementation Review
